### 서울시 공원 단계구분도

In [2]:
import numpy as np
import pandas as pd
import folium
import json
import re

- 자치구별 공원 면적 단계구분도

In [3]:
park = pd.read_csv('data/서울시 주요 공원현황.csv', encoding='euc-kr')
print(park.shape)
park.info()

(132, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            132 non-null    int64  
 1   공원명           132 non-null    object 
 2   공원개요          131 non-null    object 
 3   면적            129 non-null    object 
 4   개원일           122 non-null    object 
 5   주요시설          124 non-null    object 
 6   주요식물          88 non-null     object 
 7   안내도           102 non-null    object 
 8   오시는길          115 non-null    object 
 9   이용시참고사항       120 non-null    object 
 10  이미지           132 non-null    object 
 11  지역            131 non-null    object 
 12  공원주소          132 non-null    object 
 13  관리부서          132 non-null    object 
 14  전화번호          132 non-null    object 
 15  X좌표(GRS80TM)  122 non-null    float64
 16  Y좌표(GRS80TM)  122 non-null    float64
 17  X좌표(WGS84)    131 non-null    float64
 18  Y좌표(WGS84)    131 no

In [4]:
park = park[['면적', '공원주소',]]
park.columns = ['면적', '공원주소']
park.dropna(subset=['면적'], inplace=True)
# 과천시 삭제
# 면적 이상치 삭제
park.drop([2, 6], inplace=True)
# 총 144m 같은 데이터에서 '총  ' 지우기
park.면적 = park.면적.str.replace('총 ', '')
park.면적 = park.면적.apply(lambda x: re.split('[㎡m㎥]', x)[0])
park.면적 = park.면적.astype(float).astype(int)

In [5]:
park.isna().sum()

면적      0
공원주소    0
dtype: int64

In [6]:

park.head()

,면적,공원주소
0,2896887,서울특별시 중구 삼일대로 231(예장동)
1,80683,서울특별시 강동구 천호대로 1291(길동생태공원)
3,480994,서울특별시 성동구 뚝섬로 273 (성수동1가)
4,2284085,서울특별시 마포구 하늘공원로 84(월드컵공원)
5,8948,서울특별시 종로구 세종로 76-2


In [7]:
# 노원구로 시작하는 데이터가 있다.
park.공원주소 = park.공원주소.str.replace('서울특별시 ', '')

In [8]:
park['자치구'] = park.공원주소.apply(lambda x: x.split()[0])
park.head()

,면적,공원주소,자치구
0,2896887,중구 삼일대로 231(예장동),중구
1,80683,강동구 천호대로 1291(길동생태공원),강동구
3,480994,성동구 뚝섬로 273 (성수동1가),성동구
4,2284085,마포구 하늘공원로 84(월드컵공원),마포구
5,8948,종로구 세종로 76-2,종로구


In [9]:
spark = park.pivot_table('면적', '자치구', aggfunc='sum')
spark.head()

,면적
자치구,
강남구,3626398
강동구,1526387
강북구,2047054
강서구,2194152
관악구,10431453


In [10]:
s_file = 'data/seoul_geo_simple.json'
geo_data = json.load(open(s_file, encoding='utf-8'))

In [11]:
# 각 구별로 평균 위치 구하기
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']:
            geo = np.array(coord)
            gu_dict[gu['id']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]
    return gu_dict

gu_dict = get_text_location(geo_data)

In [12]:
# 구 이름 표시
def show_gu_name(m_park, m_dict, m_map):
    for gu_name in m_park.index:
        folium.map.Marker(
            location=m_dict[gu_name],
            icon=folium.DivIcon(icon_size=(80,20), icon_anchor=(20, 0),
                                html=f'<div style="font-size: 10pt">{gu_name}</div>')
        ).add_to(m_map)    


In [13]:
smap = folium.Map([37.55, 126.98], zoom_start=11, tiles='Stamen Toner')

folium.Choropleth(
    geo_data=geo_data,
    data=spark.면적,
    columns=[spark.index, spark.면적], 
    fill_color='PuRd',
    key_on='feature.id'
).add_to(smap)

show_gu_name(spark, gu_dict, smap)

smap

- 자치구별 인당 공원면적 단계구분도

In [14]:
pop = pd.read_csv('../02.Pandas/data/pop.csv')
pop.head()

,구별,인구수,내국인,외국인,고령자,외국인비율,고령자비율
0,종로구,153789,144683,9106,27818,5.92,18.09
1,중구,131787,122499,9288,24392,7.05,18.51
2,용산구,237285,222953,14332,39070,6.04,16.47
3,성동구,292672,285990,6682,46380,2.28,15.85
4,광진구,352627,339996,12631,51723,3.58,14.67


In [15]:
pop = pop[['구별', '인구수']]
pop.set_index('구별', inplace=True)
pop.head()


,인구수
구별,
종로구,153789
중구,131787
용산구,237285
성동구,292672
광진구,352627


In [16]:
ppark = pd.merge(pop, spark, left_index=True, right_index=True)
ppark.head()

,인구수,면적
종로구,153789,5575526
중구,131787,2975303
용산구,237285,247194
성동구,292672,626838
광진구,352627,584002


In [17]:
ppark['인당공원면적비율'] = (ppark.인구수 / ppark.면적 * 100).round(4)


In [18]:
ppark.head()

,인구수,면적,인당공원면적비율
종로구,153789,5575526,2.7583
중구,131787,2975303,4.4294
용산구,237285,247194,95.9914
성동구,292672,626838,46.6902
광진구,352627,584002,60.3811


In [19]:
smap = folium.Map([37.55, 126.98], zoom_start=11, tiles='Stamen Toner')

folium.Choropleth(
    geo_data=geo_data,
    data=ppark.인당공원면적비율,
    columns=[ppark.index, ppark.인당공원면적비율], 
    fill_color='Greens',
    key_on='feature.id'
).add_to(smap)

show_gu_name(ppark, gu_dict, smap)

smap